# Auto-Merging Retreival - Source Group A

In [9]:
import os

from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    Settings,
    load_index_from_storage,
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import HierarchicalNodeParser, get_leaf_nodes
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core.query_engine import RetrieverQueryEngine


def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    # Set global settings
    Settings.llm = llm
    Settings.embed_model = embed_model

    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)

    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes,
            storage_context=storage_context,
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
        )
    return automerging_index


def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=6,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine


In [10]:
from llama_index.llms.openai import OpenAI 
from llama_index.core import Settings

from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./SourceGroupA/source1.pdf", "./SourceGroupA/source11.pdf", "./SourceGroupA/source2.pdf", "./SourceGroupA/source3.pdf", "./SourceGroupA/source4.pdf", "./SourceGroupA/source5.pdf", "./SourceGroupA/source6.pdf", "./SourceGroupA/source7.pdf", "./SourceGroupA/source8.pdf", "./SourceGroupA/source9.pdf", "./SourceGroupA/source10.pdf"]
).load_data()


In [ ]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

index = build_automerging_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.5),
    save_dir="./merging_index_GroupA",
)


In [12]:
from llama_index.core.response.notebook_utils import display_response

query_engine = get_automerging_query_engine(index, similarity_top_k=6)

with open('./generated_questions.txt') as file:
    for line in file:
        question = line.strip()
        print(question)
        display_response(query_engine.query(question))

Are current policing methods effective in reducing crime in minority communities?
> Merging 1 nodes into parent node.
> Parent node id: 31f1f412-5bc1-4c28-9114-cc6746428b3a.
> Parent node text: There is no evidence of systematic anti-Black disparities in fatal shootings of unarmed citizens....



**`Final Response:`** Current policing methods have been subject to scrutiny regarding potential bias and racial disparities. Efforts have been made to address concerns about bias in discretionary stops and police shootings. The effectiveness of these methods in reducing crime in minority communities is a topic of ongoing debate and analysis, with some arguing that the disproportionate number of crimes committed by minorities in certain cities needs to be considered. The use of population-based statistics and comparisons to assess racial disparity or bias in policing continues to be a point of contention among proponents of police reform.

How does the use of facial recognition technology impact racial profiling?
> Merging 1 nodes into parent node.
> Parent node id: 31f1f412-5bc1-4c28-9114-cc6746428b3a.
> Parent node text: There is no evidence of systematic anti-Black disparities in fatal shootings of unarmed citizens....



**`Final Response:`** Facial recognition technology can impact racial profiling by potentially leading to biased outcomes based on the accuracy of the algorithms used. If the technology is not trained on a diverse dataset, there is a risk of misidentification, especially of individuals from minority groups. This could result in disproportionate surveillance, monitoring, or targeting of certain racial or ethnic groups, reinforcing existing biases and leading to discriminatory practices in law enforcement or other sectors where facial recognition is utilized.

Is there evidence to support claims of systemic racism within law enforcement agencies?
> Merging 1 nodes into parent node.
> Parent node id: 31f1f412-5bc1-4c28-9114-cc6746428b3a.
> Parent node text: There is no evidence of systematic anti-Black disparities in fatal shootings of unarmed citizens....



...

**`Final Response:`** Underreporting hate crimes on minority communities can lead to a lack of accurate data on the prevalence and severity of these incidents. This can result in a failure to address and combat systemic discrimination and violence against these communities effectively. Additionally, underreporting can contribute to a sense of mistrust towards law enforcement and the justice system among minority groups, potentially leading to a lack of accountability for perpetrators and a perpetuation of harmful biases and stereotypes.

How do law enforcement agencies address allegations of racial bias in their practices?


**`Final Response:`** Law enforcement agencies address allegations of racial bias in their practices by conducting research and analysis to determine if there is any systemic racial disparity or bias. They compare different data sets to assess if there is any bias in their policing data and take steps to understand the proper benchmarks for evaluating racial disparities. Additionally, they may analyze the rates of police exposure for different racial groups to ensure a comprehensive understanding of the interactions between law enforcement and the community.

What measures can be taken to ensure equitable treatment of all racial groups within the criminal justice system?


**`Final Response:`** To ensure equitable treatment of all racial groups within the criminal justice system, it is important to use appropriate benchmarks for disparity research rather than relying solely on population-based statistics. This includes considering factors such as rates of police exposure based on criminal activity rather than just overall population percentages. Additionally, efforts should be made to address any biases in the decision-making process and to analyze data sets that are uncontaminated by police bias to gain a more accurate understanding of disparities. Implementing reforms based on comprehensive and unbiased research findings can help promote fair and just treatment for all racial groups within the criminal justice system.